In [11]:
import time
import json
import random
import html
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from webdriver_manager.microsoft import EdgeChromiumDriverManager
import traceback
import logging
import re  # Make sure to include this import
from word2number import w2n
from datetime import datetime
import logging
from datetime import datetime

logging.basicConfig(level=logging.INFO)

def process_review_count(text):
    text = text.strip().replace(',', '')
    if 'K+' in text:
        return str(int(float(text.replace('(', '').replace(')', '').replace('K+', '').strip()) * 1000))
    return text

def setup_driver():
    options = webdriver.EdgeOptions()
    options.add_argument('--no-sandbox')
    try:
        driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()), options=options)
    except Exception as e:
        print(e)
        raise Exception("Failed to install Edge Chromium driver.")
    return driver



def scrape_extra_parameters(url: str, driver: webdriver.Edge) -> dict:
    try:
        driver.get(url)
        try:
            WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div[data-hook='review']")))

        except TimeoutException:
            print(f"TimeoutException: Could not find reviews for {url}")
            return {}
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # Extract the general reviews
        reviews_tags = soup.find_all('div', attrs={'data-hook': 'review'})

        result = {}
        for i, review_tag in enumerate(reviews_tags[:5]):
            result[f'Customer_{i + 1}_ID'] = review_tag.attrs.get('id', 'None')
            
            # Extract the Star Rating
            star_rating_tag = review_tag.select_one('i[data-hook="review-star-rating"] span.a-icon-alt')
            star_rating = float(star_rating_tag.text.split()[0]) if star_rating_tag else 0.0
            result[f'Customer_{i+1}_Star_Rating'] = star_rating
            
            # Extract the Comment Title
            comment_title_tag = review_tag.select_one('a[data-hook="review-title"]')
            # Inside the for loop, after extracting the comment title:
            if comment_title_tag:
                actual_comment_title = comment_title_tag.text.strip()
            else:
                # Handle alternate structure
                comment_title_tag = review_tag.select_one('span.cr-original-review-content')
                actual_comment_title = comment_title_tag.text.strip() if comment_title_tag else 'NaN'

            # Remove the pattern "k out of 5 stars\n" from the comment
            actual_comment_title = re.sub(r'\d+(\.\d+)? out of 5 stars\n', '', actual_comment_title)

            result[f'Customer_{i+1}_Comment'] = actual_comment_title

            # Extract the Number of people who found the review helpful
            helpful_vote_tag = review_tag.select_one('span[data-hook="helpful-vote-statement"]')
            helpful_count = w2n.word_to_num(helpful_vote_tag.text.split()[0]) if helpful_vote_tag else 0
            result[f'Customer_{i+1}_buying_influence'] = helpful_count
        

        # Extract Top Positive and Critical Reviews (Moved outside of the above loop)
        Parent_review_tags = soup.select('div[id^="viewpoint-"]')
        if len(Parent_review_tags) > 0: 
            ts = 'positive-review'
            result.update(extract_specific_review(Parent_review_tags[0], 'Top_Positive', ts, soup, url))

        else:
            result.update(set_default_values('Top_Positive'))
            
        if len(Parent_review_tags) > 1: 
            ts = 'critical-review.a-span-last'
            result.update(extract_specific_review(Parent_review_tags[1], 'Critical', ts, soup, url))

        else:
            result.update(set_default_values('Critical'))
            
        return result
    except Exception as e:
        print(f"Error scraping extra parameters: {e}")
        traceback.print_exc()
        return {}

def extract_specific_review(review_tag, review_type, ts, soup, url):
    specific_result = {}
    
    # Extracting ID
    review_id = review_tag.get('id', 'None').replace('viewpoint-', '')
    specific_result[f'{review_type}_Review_Cust_ID'] = review_id

    # # Extract Customer Name and Influenced
 
     # Corrected Extraction for Customer Name
    customer_name_selector = 'div.a-profile-content span.a-profile-name'
    specific_result[f'{review_type}_Review_Cust_Name'] = review_tag.select_one(customer_name_selector).text if review_tag.select_one(customer_name_selector) else 'None'

    # Corrected Selector
    influenced_selector = f'div.a-column.a-span6.view-point-review.{ts} div.a-row.a-spacing-top-small span.a-size-small.a-color-tertiary span.review-votes'
    influenced_element = soup.select_one(influenced_selector)

    if influenced_element:
        # Directly extract the text from the found element
        helpful_text = influenced_element.text.strip()
        print("Helpful Text:", helpful_text)  # Debugging line
        
        # Check if the text starts with a digit and extract the first contiguous digit sequence
        match = re.match(r'\d+', helpful_text)
        if match:
            helpful_count = int(match.group())
        else:
            # If the text doesn't start with a digit, try converting the first word to a number
            helpful_count = w2n.word_to_num(helpful_text.split()[0])
    else:
        print(f"Tag not found in {url}")  # Debugging line
        helpful_count = 0

    specific_result[f'{review_type}_Review_Cust_Influenced'] = helpful_count

    
 
  # # customer_name_tags = review_tag.select('span.a-profile-name')
    # customer_name_tags = review_tag.select('div.a-expander-content.a-expander-partial-collapse-content div.a-profile-content')
    # specific_result[f'{review_type}_Review_Cust_Name'] = customer_name_tags[1].text if len(customer_name_tags) > 1 else 'None'
    
    # Extract Customer Review Comment
    review_comment_tag = review_tag.find('div', class_='a-row a-spacing-top-mini')
    specific_result[f'{review_type}_Review_Cust_Comment'] = review_comment_tag.text.strip() if review_comment_tag else 'None'
    
    # Extract Customer Review Title
    review_title_tag = review_tag.select_one('span[data-hook="review-title"]')
    specific_result[f'{review_type}_Review_Cust_Comment_Title'] = review_title_tag.text if review_title_tag else 'None'
    
    # Extract the post time
    review_tags_date = review_tag.select('div.a-expander-content.a-expander-partial-collapse-content span.a-size-base.a-color-secondary.review-date')
    if review_tags_date:
        post_time_text = review_tags_date[0].text.strip()
        match = re.search(r'on (.+)$', post_time_text)
        if match:
            date_string = match.group(1)
            try:
                post_date = datetime.strptime(date_string, '%B %d, %Y')
                specific_result[f'{review_type}_Review_Cust_Date'] = post_date.isoformat()                            
            except ValueError as ve:
                print(f"Error parsing date string {date_string}: {ve}")
                specific_result[f'{review_type}_Review_Cust_Date'] = '-'
        else:
            print("Date not found in text:", post_time_text)
            specific_result[f'{review_type}_Review_Cust_Date'] = '-'
    else:
        print("Date tag not found")
        specific_result[f'{review_type}_Review_Cust_Date'] = None
    
    
    # Extract the Star Rating
    review_star_rating_tag = review_tag.select_one('i[data-hook="review-star-rating-view-point"] span.a-icon-alt')
    star_rating = float(review_star_rating_tag.text.split()[0]) if review_star_rating_tag else 0.0
    specific_result[f'{review_type}_Review_Cust_Star_Rating'] = star_rating
    
    return specific_result

def set_default_values(review_type):
    default_values = {
        f'{review_type}_Review_Cust_ID': 'None',
        f'{review_type}_Review_Cust_Name': 'None',
        f'{review_type}_Review_Cust_Comment': 'None',
        f'{review_type}_Review_Cust_Comment_Title': 'None',
        f'{review_type}_Review_Cust_Influenced': 0,
        f'{review_type}_Review_Cust_Star_Rating': 0.0,
        f'{review_type}_Review_Cust_Date': None,
    }
    return default_values

    #     return result
    # except Exception as e:
    #     print(f"Error scraping extra parameters for {url}: {e}")
    #     traceback.print_exc()
    # return {}

def scrape_amazon(categories):
 
    driver = setup_driver()
    all_products = []
    seen_products = set()

    for category, base_url in categories.items():
        products = []

        for page in range(1, 10):
            url = f"{base_url}&page={page}"

            try:
                driver.get(url)
                WebDriverWait(driver, 25).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#search > div.s-desktop-width-max.s-desktop-content.s-wide-grid-style-t1.s-opposite-dir.s-wide-grid-style.sg-row > div.sg-col-20-of-24.s-matching-dir.sg-col-16-of-20.sg-col.sg-col-8-of-12.sg-col-12-of-16 > div > span.rush-component.s-latency-cf-section > div.s-main-slot.s-result-list.s-search-results.sg-row > div:nth-child(1)")))
            except TimeoutException:
                print(f"Timed out waiting for elements on page {page} of category {category}.")
                continue

            time.sleep(random.uniform(3.0, 6.0))
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            
            # Find all products using the given CSS selector
            products_tags = soup.select("#search > div.s-desktop-width-max.s-desktop-content.s-wide-grid-style-t1.s-opposite-dir.s-wide-grid-style.sg-row > div.sg-col-20-of-24.s-matching-dir.sg-col-16-of-20.sg-col.sg-col-8-of-12.sg-col-12-of-16 > div > span.rush-component.s-latency-cf-section > div.s-main-slot.s-result-list.s-search-results.sg-row > div")

            products_list = []  # Use a different name for the list of product_dict dictionaries

            for product in products_tags:
                product_dict = {}
                product_dict['Product_ID'] = product.attrs.get('data-asin', None)

                # Try to find item_name with the general class
                item_name = product.find('span', class_='a-text-normal')

                # If not found, try the first specific class
                if item_name is None:
                    item_name = product.find('span', class_='a-size-base-plus a-color-base a-text-normal')

                # If still not found, try the second specific class
                if item_name is None:
                    item_name = product.find('span', class_='a-size-medium a-color-base a-text-normal')

                # If item_name is found with any class, extract the text
                if item_name:
                    product_dict['product'] = item_name.text.strip()
                else:
                    print(f"Failed to scrape item name in {url}")
                    product_dict['product'] = "Unknown"
                product_price = product.find('span', class_='a-offscreen')
                if product_price:
                    product_price = product_price.text.strip().replace("$", "").replace(",", "").strip()
                    product_dict['price'] = product_price

                rating_spans = product.find_all('span', attrs={"aria-label": True})
                for rating_span in rating_spans:
                    aria_label_value = rating_span.attrs["aria-label"]
                    if "stars" in aria_label_value:
                        product_dict['ratings'] = aria_label_value.split(" ")[0]
                    else:
                        if 'K+' in aria_label_value:
                            product_dict['review_responders'] = aria_label_value
                        else:
                            try:
                                int_value = int(aria_label_value)
                                product_dict['review_responders'] = aria_label_value
                            except ValueError:
                                pass

                item_reviews = product.find('span', class_='a-size-base s-underline-text')
                if item_reviews:
                    try:
                        reviews_text = item_reviews.text.strip()
                        reviews_count = process_review_count(reviews_text)
                        product_dict['reviews'] = reviews_count
                        logging.info(f"Successfully scraped total {product_dict['reviews']} rating for product {product_dict.get('Product_ID', 'Unknown ID')}")
                    except Exception as e:
                        logging.error(f"Error processing review count for product {product_dict.get('Product_ID', 'Unknown ID')}: {e}")
                else:
                    logging.warning(f"Failed to scrape total rating for product {product_dict.get('Product_ID', 'Unknown ID')}")

            

                # Extract ASIN
                product_dict['Product_ID'] = product.attrs.get('data-asin', None)

                # Construct the review URL using ASIN
                if product_dict['Product_ID']:
                    asin = product_dict['Product_ID']
                    product_dict['url'] = f"https://www.amazon.com/product-reviews/{asin}/ref=cm_cr_dp_d_show_all_top?ie=UTF8&reviewerType=all_reviews&sortBy=recent"
                else:
                    product_dict['url'] = "None"


                product_dict['category'] = category

                if 'Product_ID' in product_dict and product_dict['Product_ID']:
                # Create a unique identifier for the product
                    identifier = product_dict['Product_ID']

                    if identifier not in seen_products:
                        seen_products.add(identifier) #
                        if product_dict.get('url'):
                            extra_params = scrape_extra_parameters(product_dict['url'], driver)
                            product_dict.update(extra_params)
                            products_list.append(product_dict)

                        all_products.extend(products_list)
    driver.quit()
    return json.dumps(all_products)


if __name__ == '__main__':
    categories = {
        # 'Smartphones': 'https://www.amazon.com/s?k=smartphone&ref=nb_sb_noss',
        # 'Laptops': 'https://www.amazon.com/s?k=Laptops&ref=nb_sb_noss',
        # 'video_games': 'https://www.amazon.com/s?k=video_games&ref=nb_sb_noss',
        # 'Dresses':'https://www.amazon.com/s?k=Dresses&ref=nb_sb_noss',
        # 'Shoes':'https://www.amazon.com/s?k=Shoes&ref=nb_sb_noss',
        'Accessories':'https://www.amazon.com/s?k=accessories+for+clothes&ref=nb_sb_noss',
    }

    all_products = []
    try:
        all_products = json.loads(scrape_amazon(categories))
    except Exception as e:
        print(f"Error occurred during scraping: {e}")
    finally:
        with open('amazon_data_ext.json', 'w') as file:
            json.dump(all_products, file)


INFO:WDM:====== WebDriver manager ======
INFO:WDM:Get LATEST edgedriver version for Edge 117.0.2045
INFO:WDM:Get LATEST edgedriver version for Edge 117.0.2045
INFO:WDM:There is no [win64] edgedriver "117.0.2045.60" for browser edge "117.0.2045" in cache
INFO:WDM:Get LATEST edgedriver version for Edge 117.0.2045
INFO:WDM:About to download new driver from https://msedgedriver.azureedge.net/117.0.2045.60/edgedriver_win64.zip
INFO:WDM:Driver downloading response is 200
INFO:WDM:Get LATEST edgedriver version for Edge 117.0.2045
INFO:WDM:Driver has been saved in cache [C:\Users\Kasim\.wdm\drivers\edgedriver\win64\117.0.2045.60]
INFO:root:Successfully scraped total 1609 rating for product B09DXQZVP7
INFO:root:Successfully scraped total 227 rating for product B0BK59SBPJ


Tag not found in https://www.amazon.com/product-reviews/B09DXQZVP7/ref=cm_cr_dp_d_show_all_top?ie=UTF8&reviewerType=all_reviews&sortBy=recent
Tag not found in https://www.amazon.com/product-reviews/B09DXQZVP7/ref=cm_cr_dp_d_show_all_top?ie=UTF8&reviewerType=all_reviews&sortBy=recent


INFO:root:Successfully scraped total 201 rating for product B09N98H65M


Helpful Text: One person found this helpful
Tag not found in https://www.amazon.com/product-reviews/B0BK59SBPJ/ref=cm_cr_dp_d_show_all_top?ie=UTF8&reviewerType=all_reviews&sortBy=recent


INFO:root:Successfully scraped total 279 rating for product B0BK87XWF8


...

Tag not found in https://www.amazon.com/product-reviews/B0BRZ2SK7N/ref=cm_cr_dp_d_show_all_top?ie=UTF8&reviewerType=all_reviews&sortBy=recent
Tag not found in https://www.amazon.com/product-reviews/B0BRZ2SK7N/ref=cm_cr_dp_d_show_all_top?ie=UTF8&reviewerType=all_reviews&sortBy=recent


INFO:root:Successfully scraped total 30 rating for product B07MXH87VJ


Helpful Text: One person found this helpful
Tag not found in https://www.amazon.com/product-reviews/B0B97KPBPY/ref=cm_cr_dp_d_show_all_top?ie=UTF8&reviewerType=all_reviews&sortBy=recent


INFO:root:Successfully scraped total 9 rating for product B0B9GPV6FF
INFO:root:Successfully scraped total 4 rating for product B0BF8S8KX6


TimeoutException: Could not find reviews for https://www.amazon.com/product-reviews/B0BF8S8KX6/ref=cm_cr_dp_d_show_all_top?ie=UTF8&reviewerType=all_reviews&sortBy=recent
Failed to scrape item name in https://www.amazon.com/s?k=accessories+for+clothes&ref=nb_sb_noss&page=9
Failed to scrape item name in https://www.amazon.com/s?k=accessories+for+clothes&ref=nb_sb_noss&page=9
Failed to scrape item name in https://www.amazon.com/s?k=accessories+for+clothes&ref=nb_sb_noss&page=9


In [14]:
import pandas as pd
import psycopg2
import numpy as np
import re
import logging
from datetime import datetime

logging.basicConfig(level=logging.INFO)

# Load the JSON data into a pandas DataFrame
df = pd.read_json('amazon_data_ext.json')
# ---- START OF INSERTED CODE ----
def fix_products_length(df):
    """Ensure each record has a length of 42 by appending None values."""
    max_len = df.shape[1]
    if max_len < 42:
        for _ in range(42 - max_len):
            df[f'Extra_Column_{_}'] = None
    return df

# Call the function to ensure data consistency
df = fix_products_length(df)
# ---- END OF INSERTED CODE ----

# Check if specific columns are in the DataFrame
columns_to_check = ['Critical_Review_Cust_Influenced', 'Top_Positive_Review_Cust_Influenced']
for column in columns_to_check:
    if column not in df.columns:
        logging.warning(f"Column '{column}' not found in the DataFrame. Please check the column name in the JSON file.")

# Convert date columns to datetime objects and then to 'yyyy-mm-dd' string format
date_columns = ['Critical_Review_Cust_Date', 'Top_Positive_Review_Cust_Date']
for column in date_columns:
    df[column] = pd.to_datetime(df[column], errors='coerce', format='%Y-%m-%dT%H:%M:%S')
    df[column].fillna(pd.NaT, inplace=True)
    df[column] = df[column].apply(lambda x: x.strftime('%Y-%m-%d') if pd.notna(x) else '1677-09-21')

# Replace NaN values with 'None' in specific columns
columns_to_replace_nan = [
    'Critical_Review_Cust_ID', 'Critical_Review_Cust_Name', 'Critical_Review_Cust_Comment',
    'Critical_Review_Cust_Comment_Title', 'Critical_Review_Cust_Influenced',
    'Top_Positive_Review_Cust_ID', 'Top_Positive_Review_Cust_Name', 'Top_Positive_Review_Cust_Comment',
    'Top_Positive_Review_Cust_Comment_Title', 'Top_Positive_Review_Cust_Influenced'
]
for column in columns_to_replace_nan:
    df[column] = df[column].replace({np.nan: 'None'})

# Remove any duplicates that may have been created due to URL changes
df = df.drop_duplicates(subset=['Product_ID'], keep='first')

# Replace NaN values with 'None' in customer comment and ID columns
for i in range(1, 6):
    df[f'Customer_{i}_Comment'] = df[f'Customer_{i}_Comment'].replace({np.nan: 'None'})
    df[f'Customer_{i}_ID'] = df[f'Customer_{i}_ID'].replace({np.nan: 'None'})

# Define variations of NaN or missing values and replace in customer comment columns
nan_variants = [np.nan, 'NaN', 'nan', 'None', 'none', 'N/A', 'n/a', 'NA', 'na', 'null', '']
for i in range(1, 6):
    col_name = f'Customer_{i}_Comment'
    df[col_name] = df[col_name].astype(str).replace(nan_variants, 'None')

# Drop rows where specific columns have undesired values
df.dropna(subset=['price'], inplace=True)
df.drop(df.index[df['Customer_1_ID'] == 'None'], inplace=True)
df.drop(df.index[df['reviews'] == '0'], inplace=True)

# Update the 'Critical_Review_Cust_Influenced' and 'Top_Positive_Review_Cust_Influenced' columns
for column in ['Critical_Review_Cust_Influenced', 'Top_Positive_Review_Cust_Influenced']:
    df[column] = df[column].replace({'"NaN"': 0.0, 'NaN': 0.0})

# Drop the 'review_responders' column if it exists
if 'review_responders' in df.columns:
    df.drop(columns=['review_responders'], inplace=True)

# Clean other columns
df['price'] = df['price'].apply(lambda value: float(value) if isinstance(value, (int, float)) else 0.0)
df['ratings'] = df['ratings'].apply(lambda x: float(x) if pd.notna(x) else None)
df['reviews'] = df['reviews'].replace('(', '').replace(')', '').replace(nan_variants, 0).fillna(0).astype(int)



# Connect to PostgreSQL
conn = psycopg2.connect(
    host="localhost",
    database="postgres",
    user="postgres",
    password="demopass",
    client_encoding='utf8'
)
cur = conn.cursor()

# Modify the CREATE TABLE query to include additional columns
create_table_query = """
DROP TABLE IF EXISTS amazon_data_ext;
CREATE TABLE IF NOT EXISTS amazon_data_ext (
    Product_ID TEXT NOT NULL,
    product TEXT NOT NULL,
    price NUMERIC NULL,
    ratings NUMERIC NULL,
    reviews INTEGER NOT NULL,
    category TEXT NOT NULL,
    url TEXT NOT NULL,
    Top_Positive_Review_Cust_ID TEXT,
    Top_Positive_Review_Cust_Name TEXT,
    Top_Positive_Review_Cust_Date DATE,
    Top_Positive_Review_Cust_Comment TEXT,
    Top_Positive_Review_Cust_Comment_Title TEXT,
    Top_Positive_Review_Cust_Influenced INTEGER,
    Top_Positive_Review_Cust_Star_Rating NUMERIC,
    Critical_Review_Cust_ID TEXT,
    Critical_Review_Cust_Name TEXT,
    Critical_Review_Cust_Date DATE,
    Critical_Review_Cust_Comment TEXT,
    Critical_Review_Cust_Comment_Title TEXT,
    Critical_Review_Cust_Influenced INTEGER,
    Critical_Review_Cust_Star_Rating NUMERIC,
    """ + ",\n    ".join([f"Customer_{i}_ID TEXT, Customer_{i}_Star_Rating NUMERIC, Customer_{i}_Comment TEXT, Customer_{i}_buying_influence INTEGER" for i in range(1, 6)]) + """
)
"""
cur.execute(create_table_query)
conn.commit()

def clean_format_data(row):
    # Extract values directly, as they are already cleaned
    ratings = row['ratings']
    price = row['price']
    reviews = row['reviews']
    product_id = row['Product_ID']
    product = row['product']
    category = row['category']
    url = row['url']
      
    critical_review_id = row['Critical_Review_Cust_ID'] if row['Critical_Review_Cust_ID'] != 'None' else None
    critical_review_cust_name = row['Critical_Review_Cust_Name'] if row['Critical_Review_Cust_Name'] != 'None' else None
    critical_review_cust_comment = row['Critical_Review_Cust_Comment']
    if critical_review_cust_comment in nan_variants or critical_review_cust_comment == 'None':
        critical_review_cust_comment = 'Unavailable'
    
    critical_review_cust_comment_title = row['Critical_Review_Cust_Comment_Title'] if row['Critical_Review_Cust_Comment_Title'] != 'None' else None
    critical_review_cust_influenced = row['Critical_Review_Cust_Influenced'] if row['Critical_Review_Cust_Influenced'] != 'None' else 0  # Correctly handle NaN values
    critical_review_star_rating = row['Critical_Review_Cust_Star_Rating'] if pd.notna(row['Critical_Review_Cust_Star_Rating']) else 0.0
    critical_review_cust_date = row['Critical_Review_Cust_Date'] if row['Critical_Review_Cust_Date'] != 'None' else '0001-01-01'  # Correctly handle NaN values

    top_positive_review_id = row['Top_Positive_Review_Cust_ID'] if row['Top_Positive_Review_Cust_ID'] != 'None' else None
    top_positive_review_cust_name = row['Top_Positive_Review_Cust_Name'] if row['Top_Positive_Review_Cust_Name'] != 'None' else None
    top_positive_review_cust_comment = row['Critical_Review_Cust_Comment']
    if top_positive_review_cust_comment in nan_variants or top_positive_review_cust_comment == 'None':
        top_positive_review_cust_comment = None    
    top_positive_review_cust_comment_title = row['Top_Positive_Review_Cust_Comment_Title'] if row['Top_Positive_Review_Cust_Comment_Title'] != 'None' else None
    top_positive_review_cust_influenced = row['Top_Positive_Review_Cust_Influenced'] if row['Top_Positive_Review_Cust_Influenced'] != 'None' else 0  # Correctly handle NaN values
    top_positive_review_star_rating = row['Top_Positive_Review_Cust_Star_Rating'] if pd.notna(row['Top_Positive_Review_Cust_Star_Rating']) else 0.0
    top_positive_review_cust_date = row['Top_Positive_Review_Cust_Date'] if row['Top_Positive_Review_Cust_Date'] != 'None' else '0001-01-01'  # Correctly handle NaN values
    
    top_positive_date = row['Top_Positive_Review_Cust_Date']
    critical_review_date = row['Critical_Review_Cust_Date']

    def format_date(date_value):
        if isinstance(date_value, str) and re.match(r'\d{4}-\d{2}-\d{2}', date_value):
            return date_value
        else:
            return '0001-01-01'  # Default value for invalid date formats or non-string values

    top_positive_review_cust_date = format_date(top_positive_date)
    critical_review_cust_date = format_date(critical_review_date)



    # Handle additional customer information
    customer_data = []
    for i in range(1, 6):
        customer_id = row[f'Customer_{i}_ID'] if row[f'Customer_{i}_ID'] != 'None' else "Unavailable"
        star_rating = row[f'Customer_{i}_Star_Rating'] if pd.notna(row[f'Customer_{i}_Star_Rating']) else 0.0
        comment = row[f'Customer_{i}_Comment'] if row[f'Customer_{i}_Comment'] != 'None' else "Unavailable"
        buying_influence = row[f'Customer_{i}_buying_influence'] if pd.notna(row[f'Customer_{i}_buying_influence']) else 0
        customer_data.extend([customer_id, star_rating, comment, buying_influence])

    # Construct the return tuple
    result_tuple = (product_id, product, price, ratings, reviews, category, url, 
                   top_positive_review_id, top_positive_review_cust_name, top_positive_review_cust_date, 
                   top_positive_review_cust_comment, top_positive_review_cust_comment_title, 
                   top_positive_review_cust_influenced, top_positive_review_star_rating, 
                   critical_review_id, critical_review_cust_name, critical_review_cust_date, critical_review_cust_comment, 
                   critical_review_cust_comment_title, critical_review_cust_influenced, 
                   critical_review_star_rating, *customer_data)
    
    if not result_tuple:
        logging.error(f"Failed to construct tuple for row: {row}")
        return None
    
    return result_tuple

# Check for the presence of the column `Customer_{i}_buying_influence` in the DataFrame
for i in range(1, 6):
    if f'Customer_{i}_buying_influence' not in df.columns:
        logging.error(f"Column 'Customer_{i}_buying_influence' not found in the DataFrame.")


# Define the INSERT query
insert_query = """
INSERT INTO amazon_data_ext (
    Product_ID, product, price, ratings, reviews, category, url,
    Top_Positive_Review_Cust_ID, Top_Positive_Review_Cust_Name, Top_Positive_Review_Cust_Date, Top_Positive_Review_Cust_Comment, Top_Positive_Review_Cust_Comment_Title, Top_Positive_Review_Cust_Influenced, Top_Positive_Review_Cust_Star_Rating, Critical_Review_Cust_ID, Critical_Review_Cust_Name, Critical_Review_Cust_Date, Critical_Review_Cust_Comment, Critical_Review_Cust_Comment_Title, Critical_Review_Cust_Influenced, Critical_Review_Cust_Star_Rating,
    """ + ", ".join([f"Customer_{i}_ID, Customer_{i}_Star_Rating, Customer_{i}_Comment, Customer_{i}_buying_influence" for i in range(1, 6)]) + """
) VALUES (""" + ", ".join(["%s"] * (21 + 20)) + ")"


# Count the number of placeholders in the SQL query
num_placeholders = insert_query.count('%s')

for index, row in df.iterrows():
    tuple_values = clean_format_data(row)
    if not tuple_values:
        logging.warning(f"Skipping row at index {index} due to errors in data processing.")
        continue
    num_tuple_values = len(tuple_values)
    
    # Check for mismatch between placeholders and tuple values
    if num_placeholders != num_tuple_values:
        logging.error(f"Mismatch at index {index}! Number of placeholders: {num_placeholders}, Number of tuple values: {num_tuple_values}")
        logging.error(f"Tuple values: {tuple_values}")
        
        # Expected columns based on the INSERT query
        expected_columns = [
            "Product_ID", "product", "price", "ratings", "reviews", "category", "url",
            "Top_Positive_Review_Cust_ID", "Top_Positive_Review_Cust_Name", "Top_Positive_Review_Cust_Date", "Top_Positive_Review_Cust_Comment", "Top_Positive_Review_Cust_Comment_Title", "Top_Positive_Review_Cust_Influenced",
            "Top_Positive_Review_Cust_Star_Rating", "Critical_Review_Cust_ID", "Critical_Review_Cust_Name", "Critical_Review_Cust_Date", "Critical_Review_Cust_Comment", "Critical_Review_Cust_Comment_Title",
            "Critical_Review_Cust_Influenced", "Critical_Review_Cust_Star_Rating"
        ] + [f"Customer_{i}_ID" for i in range(1, 6)] + [f"Customer_{i}_Star_Rating" for i in range(1, 6)] + [f"Customer_{i}_Comment" for i in range(1, 6)] + [f"Customer_{i}_buying_influence" for i in range(1, 6)]



        # In the section where you're logging the mismatch error, add this:
        for col, val in zip(expected_columns, tuple_values):
            print(f"{col}: {val}")

        
        continue  # Skip this iteration


    try:
        cur.execute(insert_query, tuple_values)
    except Exception as e:
        logging.error(f"Error inserting row at index {index}: {e}")
        logging.debug(f"Row data: {row}")
        conn.rollback()

        

conn.commit()
cur.close()
conn.close()
# Rename the columns in the DataFrame
df.rename(columns={'ratings': 'star_ratings', 'reviews': 'total_ratings', 'price': 'price_dollars'}, inplace=True)

# Save the DataFrame to a CSV file with updated column names
df.to_csv('amazon_data_ext.csv', index=False, encoding='utf-8')